## A simple self-attention mechanism without trainable weights

In [1]:
import torch

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your (x1)
        [0.55, 0.87, 0.66], # Jounery (x2)
        [0.57, 0.85, 0.64], # starts (x3)
        [0.22, 0.58, 0.33], # with (x4)
        [0.77, 0.25, 0.1], # one (x5)
        [0.05, 0.8, 0.55] # step (x6)
    ]
)

In [2]:
#1 The second input token serves as the query.

query = inputs[1] # 1 get "Jounery" as a query
print(query)

tensor([0.5500, 0.8700, 0.6600])


In [3]:
# generate a tensor given by the input dim
attn_scores_2 = torch.empty(inputs.shape[0]) 

# iterate each word in input and compute attention scores between query token and other tokens
for i, x_i in enumerate(inputs): 
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


## Normalizing attentionn scores 

In [4]:
# Understanding dot product
res = 0.
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


In [5]:
# unstable approach
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights: ", attn_weights_2_tmp)
print("Sum: ", attn_weights_2_tmp.sum())

Attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:  tensor(1.0000)


In [6]:
# stable approach
def softmax_navie(x):
    return torch.exp(x) / torch.exp(x).sum(dim = 0)

attn_weights_2_navie = softmax_navie(attn_scores_2)
print("Attention weights: ", attn_weights_2_navie)
print("Sum: ", attn_weights_2_navie.sum())

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


In [7]:
# More stable approach
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)
print("Attention weights: ", attn_weights_2)
print("Sum: ", attn_weights_2.sum())

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


## Computing context vector

In [8]:
# context vector

# second query
query = inputs[1] 

# initialize a tensor with all zero values
context_vec_2 = torch.zeros(query.shape)

# compute context vector
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## Computing attention weights for all input tokens

In [9]:
# iterative approach

# initailize a 6 by 6 tensor
attn_scores = torch.empty(6, 6)

# Obtain a query
for i, x_i in enumerate(inputs):
    # claculate attention score by query tokens and other tokens 
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
# vectorization approach
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [11]:
# apply softmax normalization to achieve attention weights
attn_weights = torch.softmax(attn_scores, dim = -1)

print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum: ", row_2_sum)
print("All row sums: ", attn_weights.sum(dim=-1))

Row 2 sum:  1.0
All row sums:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [13]:
# obtain all context vectors as a matrix by attention weights and inputs 
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## Implementing self-attention with trainable weights

In [15]:
#1 The second input element
#2 The input embedding size, d=3
#3 The output embedding size, d_out=2

x_2 = inputs[1] # 1
d_in = inputs.shape[1] # 2
d_out = 2 # 3

In [16]:
x_2

tensor([0.5500, 0.8700, 0.6600])

In [17]:
# requires_grad = False to reduce clutter in the outputs, 
# but if we were to use the weight matrices for model training, 
# we would set requires_grad=True to update these matrices during model training.

torch.manual_seed(123)

# initialize weight matrices by random values
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [18]:
torch.rand(d_in, d_out)

tensor([[0.3821, 0.6605],
        [0.8536, 0.5932],
        [0.6367, 0.9826]])

In [19]:
torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

Parameter containing:
tensor([[0.2745, 0.6584],
        [0.2775, 0.8573],
        [0.8993, 0.0390]])

In [20]:
# Achieve query, key, and value by matrix multiplication 
#   between input 2 and weight matrices
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [21]:
# weight matrices
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape: ", keys.shape)
print("values.shape: ", values.shape)

keys.shape:  torch.Size([6, 2])
values.shape:  torch.Size([6, 2])


In [22]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])

In [23]:
# query @ keys = attention scores 

#1 Remember that Python starts indexing at 0.

keys_2 = keys[1] # 1
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [24]:
#1 All attention scores for given query

attn_scores_2 = query_2 @ keys.T # 1
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [25]:
# Attention weight / scaled dot product attention
# reduce dimension of the matrix

# The scaling by the square root of the embedding dimension is to  avoiding small gradients
#  and is the reason why this self-attention mechanism is also called scaled-dot product attention.

d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim = -1)
print(d_k)
print(attn_weights_2)

2
tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [26]:
# context vector z = attention weights @ values
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


## Implementing a compcact self-attention Python class 

In [27]:
import torch.nn as nn
# SelfAttention_v1 is a class derived from nn.Modulewhen 
# you define a custom module (like SelfAttention_v1) that inherits from nn.Module, 
# the forward method is automatically invoked when you call the module instance.

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        # initialize a tensor given by input dim x output dim and convert tensor to parameter
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega
        
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

In [28]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [29]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        # nn.Linear = nn.Parameter(torch.rand())^T
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

In [30]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## The relationship between nn.linear and nn.Parameter

In [31]:
# Transfer weights
def transfer_weights(v2, v1):
    # Extract weights and bias from SelfAttention_v2
    weight_v2 = v2.W_query.weight.T 
    v1.W_query.data = weight_v2

    weight_v2 = v2.W_key.weight.T 
    v1.W_key.data = weight_v2

    weight_v2 = v2.W_value.weight.T 
    v1.W_value.data = weight_v2

In [32]:
v2 = SelfAttention_v2(d_in, d_out)
v1 = SelfAttention_v1(d_in, d_out)

# Transfer weights from v2 to v1
transfer_weights(v2, v1)

# Check that both produce the same results
output_v2 = v2(inputs)  # From SelfAttention_v2
output_v1 = v1(inputs)  # From SelfAttention_v1

In [33]:
print(output_v2)
print(output_v1)

tensor([[ 0.3671, -0.3086],
        [ 0.3675, -0.3095],
        [ 0.3675, -0.3094],
        [ 0.3670, -0.3073],
        [ 0.3670, -0.3061],
        [ 0.3672, -0.3085]], grad_fn=<MmBackward0>)
tensor([[ 0.3671, -0.3086],
        [ 0.3675, -0.3095],
        [ 0.3675, -0.3094],
        [ 0.3670, -0.3073],
        [ 0.3670, -0.3061],
        [ 0.3672, -0.3085]], grad_fn=<MmBackward0>)


In [34]:
torch.allclose(output_v1, output_v2, atol=1e-6)

True

## Applying a causal attention mask

In [35]:
#1 Reuses the query and  key weight matrices of 
# the SelfAttention_v2 object from the previous section for convenience

queries = sa_v2.W_query(inputs) # 1
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [36]:
# use tril function to create a mask

# attention score is a squared matrix
context_length = attn_scores.shape[0]

# initialize a tensor with all 1 values given by context_length by context_length dimension
# masked the upper diganoal values by 0
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [37]:
# perform a normal matrix multiplication to mask the upper digaonal values in attn_weights
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [38]:
row_sums = masked_simple.sum(dim = -1, keepdim=True)

# normalize values after masking
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [39]:
# Information Leakage

# softmax(-inf) = 0

# torch.triu() map upper diganoal value with True
mask  = torch.triu(torch.ones(context_length, context_length), diagonal=1)

# masked_fill() if value is true, map True value with -inf
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)

print(attn_scores)
print(mask.bool())
print(mask)
print(masked)

tensor([[ 0.2899,  0.0716,  0.0760, -0.0138,  0.1344, -0.0511],
        [ 0.4656,  0.1723,  0.1751,  0.0259,  0.1771,  0.0085],
        [ 0.4594,  0.1703,  0.1731,  0.0259,  0.1745,  0.0090],
        [ 0.2642,  0.1024,  0.1036,  0.0186,  0.0973,  0.0122],
        [ 0.2183,  0.0874,  0.0882,  0.0177,  0.0786,  0.0144],
        [ 0.3408,  0.1270,  0.1290,  0.0198,  0.1290,  0.0078]],
       grad_fn=<MmBackward0>)
tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])
tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [

In [40]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


## Masking additional attention weights with dropout

In [41]:
#1 We choose a dropout rate of 50%.
#2 Here, we create a matrix of 1s.

torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # 1
example = torch.ones(6, 6) # 2
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [42]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


## Impelmenting a compact causal attention class

In [43]:
#1 Two inputs with six tokens each; each token has embedding dimension 3

batch = torch.stack((inputs, inputs), dim = 0)
print(batch.shape)

torch.Size([2, 6, 3])


In [44]:
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [60]:
#1 Compared to the previous SelfAttention_v1 class, we added a dropout layer.
#2 The register_buffer call is also a new addition (more information is provided in the following text).
#3 We transpose dimensions 1 and 2, keeping the batch dimension at the first position (0).
#4 In PyTorch, operations with a trailing underscore are performed in-place, 
# avoiding unnecessary memory copies.

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout,
                 qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)  #1
        self.register_buffer(
            "mask",
            # context_length determines the matrix is a squared matrix
            torch.triu(torch.ones(context_length, context_length), diagonal=1),
        ) # 2

    def forward(self, x):
        # batch, row, column
        b, num_tokens, d_in = x.shape #3
        
        keys = self.W_key(x)
        print(keys)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        # transpose row dim and column dim but keep the batch dim 
        attn_scores = queries @ keys.transpose(1, 2) # omega
        print(keys.transpose(1, 2))
        attn_scores.masked_fill_( # 4
            self.mask.bool()[:num_tokens, :num_tokens], 
            -torch.inf
        )

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [61]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape: ", context_vecs.shape)

tensor([[[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]],

        [[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]]], grad_fn=<UnsafeViewBackward0>)
tensor([[[-0.5740, -0.8709, -0.8628, -0.4789, -0.4744, -0.5888],
         [ 0.2727,  0.1008,  0.1060,  0.0051,  0.1696, -0.0388]],

        [[-0.5740, -0.8709, -0.8628, -0.4789, -0.4744, -0.5888],
         [ 0.2727,  0.1008,  0.1060,  0.0051,  0.1696, -0.0388]]],
       grad_fn=<TransposeBackward0>)
context_vecs.shape:  torch.Size([2, 6, 2])


## Stacking multiple single-head attention layers

In [47]:
# Multiple CausalAttention

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim = -1)


In [48]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2

# num_heads = 2 means we have two casual attentions
# context length = input length
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads = 2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape: ", context_vecs.shape)

tensor([[[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]],

        [[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]]], grad_fn=<UnsafeViewBackward0>)
tensor([[[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]],

        [[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]]], grad_fn=<UnsafeViewBackward0>)
tensor([[[-0.3132, -0.2272],
         [-0.1536,  0.2768],
         [-0.1574,  0.2865],
         [-0.0360,  0.1826],
         [-0.1805,  0.3798],
         [-0.0080,  0.0967]],

        [[-0.3132, -0.2272],
         [-0.1536,  0.2768],


## Implementing multi-head attention with weight splits

In [49]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout,
                 num_heads, qkv_bias=False):
        super().__init__()
        assert(d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads

        #1 Reduces the projection dim to match the desired output dim
        self.head_dim = d_out // num_heads # 1

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        #2 Uses a Linear layer to combine head outputs
        self.out_proj = nn.Linear(d_out, d_out) # 2

        self.dropout = nn.Dropout(dropout) 
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1),
        ) 

    def forward(self, x):
        #3 Tensor shape: (b, num_tokens, d_out)
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # 3
        queries = self.W_query(x) # 3
        values = self.W_value(x) # 3

        #4 We implicitly split the matrix by adding a num_heads dimension. 
        # Then we unroll the last dim: (b, num_tokens, d_out) 
        # -&gt; (b, num_tokens, num_heads, head_dim).
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) # 4
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        #5 Transposes from shape (b, num_tokens, num_heads, head_dim) 
        # to (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2) # 5
        queries = queries.transpose(1, 2) # 5
        values = values.transpose(1, 2) # 5

        #6 Computes dot product for each head
        attn_scores = queries @ keys.transpose(2, 3) # omega # 6

        #7 Masks truncated to the number of tokens
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # 7

        #8 Uses the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf) # 8

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        #9 Tensor shape: (b, num_tokens, n_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) # 9

        #10 Combines heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view( # 10
            b, num_tokens, self.d_out
        )

        #11 Adds an optional linear projection
        context_vec = self.out_proj(context_vec) # 11
        return context_vec

### Batched matrix Multiplication

In [50]:
x = torch.randn(4, 4)
x.size()
y = x.view(16)

z = x.view(-1, 8)  # the size -1 is inferred from other dimensions

print(x)
print(y)
print(z)

tensor([[-0.3885, -0.9343, -0.4991, -1.0867],
        [ 0.9624,  0.2492, -0.4845, -2.0929],
        [ 0.0983, -0.0935,  0.2662, -0.5850],
        [-0.3430, -0.6821, -0.9887, -1.7018]])
tensor([-0.3885, -0.9343, -0.4991, -1.0867,  0.9624,  0.2492, -0.4845, -2.0929,
         0.0983, -0.0935,  0.2662, -0.5850, -0.3430, -0.6821, -0.9887, -1.7018])
tensor([[-0.3885, -0.9343, -0.4991, -1.0867,  0.9624,  0.2492, -0.4845, -2.0929],
        [ 0.0983, -0.0935,  0.2662, -0.5850, -0.3430, -0.6821, -0.9887, -1.7018]])


In [51]:
a = torch.randn(1, 2, 3, 4)
b = a.transpose(1, 2)  # Swaps 2nd and 3rd dimension
c = a.view(1, 3, 2, 4)  # Does not change tensor layout in memory

print(a)

tensor([[[[-1.2203,  1.3139,  1.0533,  0.1388],
          [-0.2044, -2.2685, -0.9133, -0.4204],
          [ 0.2436, -0.0567,  0.3784,  1.6863]],

         [[ 0.2553, -0.5496,  1.0042,  0.8272],
          [ 1.5434,  0.1406,  1.0617, -0.9929],
          [-1.6025, -1.0764,  0.9031, -0.7218]]]])


In [52]:
print(b)

tensor([[[[-1.2203,  1.3139,  1.0533,  0.1388],
          [ 0.2553, -0.5496,  1.0042,  0.8272]],

         [[-0.2044, -2.2685, -0.9133, -0.4204],
          [ 1.5434,  0.1406,  1.0617, -0.9929]],

         [[ 0.2436, -0.0567,  0.3784,  1.6863],
          [-1.6025, -1.0764,  0.9031, -0.7218]]]])


In [53]:
print(c)

tensor([[[[-1.2203,  1.3139,  1.0533,  0.1388],
          [-0.2044, -2.2685, -0.9133, -0.4204]],

         [[ 0.2436, -0.0567,  0.3784,  1.6863],
          [ 0.2553, -0.5496,  1.0042,  0.8272]],

         [[ 1.5434,  0.1406,  1.0617, -0.9929],
          [-1.6025, -1.0764,  0.9031, -0.7218]]]])


In [54]:
#1 The shape of this tensor is (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4).

a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],    #1
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

In [55]:
a.transpose(2, 3)

tensor([[[[0.2745, 0.8993, 0.7179],
          [0.6584, 0.0390, 0.7058],
          [0.2775, 0.9268, 0.9156],
          [0.8573, 0.7388, 0.4340]],

         [[0.0772, 0.4066, 0.4606],
          [0.3565, 0.2318, 0.5159],
          [0.1479, 0.4545, 0.4220],
          [0.5331, 0.9737, 0.5786]]]])

In [56]:
print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


In [57]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print(first_head)
print("\nFirst head: \n", first_res)

tensor([[0.2745, 0.6584, 0.2775, 0.8573],
        [0.8993, 0.0390, 0.9268, 0.7388],
        [0.7179, 0.7058, 0.9156, 0.4340]])

First head: 
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])


In [58]:
second_head = a[0, 1, :, :]
print(second_head)
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

tensor([[0.0772, 0.3565, 0.1479, 0.5331],
        [0.4066, 0.2318, 0.4545, 0.9737],
        [0.4606, 0.5159, 0.4220, 0.5786]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [59]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
